# Terraform Modules

> *Terraform modules* are reusable and shareable packages of Terraform configurations that represent a specific piece of infrastructure or a particular use case. They allow you to abstract and standardize your infrastructure code into manageable components, which can be reused across different projects or configurations.

For example, consider a scenario where you need to deploy an Azure Virtual Network (VNet) with associated subnets, network security groups, and route tables. Instead of writing the same network configuration repeatedly, you can create a Terraform module that encapsulates this network infrastructure. You can then use this module whenever you need to create a new project that needs a VNet.

IMPORTANT: Any resources you provision on Azure will accrue a cost. With an Azure Free Tier account you are allowed a certain capacity free for each resource, which you can learn more about here: [Azure Free Tier account services](https://azure.microsoft.com/en-gb/free/search/?ef_id=_k_CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE_k_&OCID=AIDcmmiouhop3i_SEM__k_CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE_k_&gad_source=1&gclid=CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE).
Once you run out of credits on your free tier subscription you will be charged, so make sure to decomission any resources you create on this account once you are finished with them.

## Module Structure and Organization

Terraform modules typically have a defined structure and organization to ensure clarity and maintainability. Here's a common structure:

- **Module Directory**: Each module is typically organized within its own directory. This directory contains the module's Terraform configuration files (`.tf` files) and any other necessary assets.

- **Main Configuration File**: The main configuration file, often named `main.tf`, defines the infrastructure resources and their configurations

- **Input Variables File**: The `variables.tf` file is used to define input variables for the module. Input variables allow users to customize the module's behavior when they use it in their configurations.

- **Variable Definitions File**: The `terraform.tfvars` file is used to set values for the input variables defined in `variables.tf`. This file provides a convenient way to define variable values without hardcoding them into the configuration files. It allows you to separate variable values from your code, making your module more reusable and easier to manage.

> Typically, the `terraform.tfvars` file contains sensitive information or environment-specific configurations that should not be exposed publicly. Therefore, it is not recommended to upload this file to GitHub! So the file `terraform.tfvars` should be included in `.gitignore`.

- **Output Values File**: The `outputs.tf` file declares output values that provide information about the resources created by the module. Output values enable data to flow from the module back to the calling configuration.

Here's an example structure of a Terraform mdoule:

```hcl
my-module/
├── main.tf
├── variables.tf
├── terraform.tfvars
├── outputs.tf

```

## Input and Output Variables

Input and output variables are fundamental to Terraform modules. They provide a way to customize the behavior of a module and retrieve information from it.

*Input variables* are parameters that allow users of a module to configure it according to their specific requirements. By defining input variables, modules become flexible and adaptable to various scenarios. Let's look at an example input variable for an Azure Windows Virtual Machine module:

```hcl
# variables.tf

variable "vm_name" {
  description = "The name of the Azure Windows Virtual Machine."
  type        = string
}

variable "vm_size" {
  description = "The size of the Azure Windows Virtual Machine."
  type        = string
}
```

In this example, the module defines input variables for specifying the VM name and size when using the module.

*Output values*, on the other hand, allow modules to communicate information or status back to the calling configuration. Users can access these values and use them in their Terraform code. Let's look at an example output variable for an Azure Windows Virtual Machine module:

```hcl
# outputs.tf

output "vm_public_ip" {
  description = "The public IP address of the Azure Windows Virtual Machine."
  value       = azurerm_windows_virtual_machine.example.public_ip_address
}
```
In this example, the module declares an output value for the VM's public IP address, which can be retrieved and used in the calling configuration.

With input and output variables, users can customize modules and retrieve important information, making modules highly reusable and versatile.

## Hands-On: Building a Basic Module

In this example, we'll create a Terraform module that deploys an Azure Storage Account. The module will allow users to customize the storage account's name, location, tier and replication type.

### Step 1: Module Directory Structure

Begin by creating a project folder where you'll organize your Terraform module. Choose a meaningful name for your project, e.g., `azure_storage_project`. This project folder will contain both your module and the main configuration files.

Within your project folder (`azure_storage_project`), set up the directory structure for your Terraform module. Create a directory for your module, e.g., `azure_storage_module`. Inside this directory, you should have the following files:

- `main.tf`: This file will contain the main Terraform configurations for creating the VM
- `variables.tf`: Here, you'll define input variables that allow users to customize the VM
- `outputs.tf`: In this file, you'll declare output values to provide information about the created VM

Your directory structure should look like this:

```css
azure_storage_project/
│
└── azure_storage_module/
    │   main.tf
    │   variables.tf
    │   outputs.tf

```

### Step 2: Input Variables

In the `variables.tf` file, you define input variables that users of your module can customize. Input variables make your module flexible and adaptable to different use cases.

```hcl
# variables.tf

variable "storage_account_name" {
  description = "The name of the Azure Storage Account."
  type        = string
}

variable "resource_group_name" {
  description = "The name of the Azure resource group where the Storage Account will be created."
  type        = string
}

variable "location" {
  description = "The Azure region where the Storage Account will be deployed."
  type        = string
  default     = "East US"  # Default to East US region
}

variable "account_tier" {
  description = "The storage account performance tier (Standard or Premium)."
  type        = string
  default     = "Standard"
}

variable "account_replication_type" {
  description = "The replication type for the Storage Account (LRS, GRS, ZRS, or RAGRS)."
  type        = string
  default     = "LRS"
}
```

Notice that in this example the variables `location`, `account_tier` and `account_replication_type` have default values provided

> While defaults make modules easy to use, there are cases where you might want to define variables without defaults in `variables.tf` and provide their values in `terraform.tfvars`. This is very important when dealing with sensitive information or environment-specific configurations.

### Step 3: Resource Configuration

Now, let's define the Azure Storage Account resource in the `main.tf` file using the input variables:

```hcl
# main.tf

# Configure the Azure provider
terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "=3.95.0"
    }
  }
}

provider "azurerm" {
  features {}
  skip_provider_registration = true
  client_id       = "your-client-id"
  client_secret   = "your-client-secret"
  subscription_id = "your-subscription-id"
  tenant_id       = "your-tenant-id"
}

# Create a resource group for the Storage Account
resource "azurerm_resource_group" "example" {
  name     = var.resource_group_name
  location = var.location
}

# Define the Azure Storage Account
resource "azurerm_storage_account" "example" {
  name                     = var.storage_account_name
  resource_group_name      = azurerm_resource_group.example.name
  location                 = var.location
  account_tier             = var.account_tier
  account_replication_type = var.account_replication_type
  # Add a dependency on the resource group
  depends_on = [azurerm_resource_group.example]
}

```

In the provided Terraform configuration, an Azure Storage Account resource is defined using the `azurerm_storage_account` block. Here's a brief explanation of the Azure Storage Account resource and its properties:

- `resource "azurerm_storage_account" "example"`: This declares an Azure Storage Account resource with the name `"example"`. You can refer to this resource using `azurerm_storage_account.example`.

- `name`: This property specifies the name of the Azure Storage Account. The value is set to the input variable `var.storage_account_name`, which allows users to provide a custom name for the account.

- `resource_group_name`: This property defines the name of the Azure resource group to which the Storage Account should be associated. It references the name property of the `azurerm_resource_group.example` resource, ensuring that the Storage Account is created within that resource group.

- `location`: This property sets the Azure region where the Storage Account will be deployed. It is based on the input variable `var.location`, which users can customize.

- `account_tier`: This property allows users to specify the performance tier for the Storage Account. It references the input variable `var.account_tier`, and users can choose between `"Standard"` or `"Premium"` for the tier.

- `account_replication_type`: Users can define the replication type for the Storage Account using this property. It references the input variable `var.account_replication_type`, and options include `"LRS"` (Locally Redundant Storage), `"GRS"` (Geo-Redundant Storage), `"ZRS"` (Zone-Redundant Storage), or `"RAGRS"` (Read-Access Geo-Redundant Storage).

- `depends_on`: This property ensures that the creation of the Azure Storage Account is dependent on the successful creation of the Azure resource group (`azurerm_resource_group.example`). It establishes a dependency relationship to guarantee the resource group is created before the Storage Account.

### Step 4: Output Values

In the `outputs.tf` file, declare output values to provide information about the created Storage Account:

```hcl
# outputs.tf

output "storage_account_id" {
  description = "The ID of the created Azure Storage Account."
  value       = azurerm_storage_account.example.id
}
```

- `azurerm_storage_account.example` references the Azure Storage Account resource you've defined in your Terraform configuration. Specifically, it points to the instance of that resource named `"example"`.

- `.id` is used to access a specific property of the Azure Storage Account resource. In this case, it's retrieving the id property.

### Step 5: Initialize the Module

Before you can use the module, you need to initialize it. In your module directory (`azure_storage_module`), run the `terraform init` command.

### Step 6: Using the Module

Now that you've defined your module and initialized it, you can use it in a configuration. Create a new Terraform configuration file (e.g., `storage_account.tf`) within your project directory (`azure_storage_project`) and use the module to provision a Storage Account:

```hcl
# storage_account.tf

module "storage_account" {
  source                 = "./azure_storage_module"
  storage_account_name   = "<unique-storage-name>"    # Customize the name !! This has to be unique name
  resource_group_name    = "myresourcegroup"     # Customize the resource group name
  location               = "East US"             # Customize the region
  account_tier           = "Standard"            # Customize the tier
  account_replication_type = "LRS"               # Customize the replication type
}

output "storage_account_id" {
  value = module.storage_account.storage_account_id
}
```

- `output "storage_account_id"`: This defines an output variable named `"storage_account_id"`. It specifies that the value of this output is equal to the `storage_account_id` output variable from the `azure_storage_module`.

- `value = module.storage_account.storage_account_id`: This line specifies that the `"storage_account_id"` output variable should capture the value of the `storage_account_id` output from the `azure_storage_module`.

### Step 7: Apply the Configuration

To create the Azure Storage Account, first run the `terraform init` command within your project directory `azure_storage_project` and then run `terraform apply`. Terraform will display the changes it intends to make. If everything looks correct, confirm by typing **yes** when prompted. Terraform will proceed to create the Storage Account and the associated resources based on your configuration.

### Step 8: View Outputs (Optional)

After applying the configuration, you can view the output values, such as the Storage Account name running the following command: `terraform output storage_account_id`. This will display the name of the created Storage Account.

### Step 9: Managing Your Infrastructure

You've successfully created a Terraform module for provisioning an Azure Storage Account. You can now manage your infrastructure using Terraform. If you need to make changes, simply modify your configuration files and run `terraform apply` again to apply those changes.

With this modular approach, you can easily reuse this module in other projects and environments by providing different values for the input variables, making infrastructure management more efficient and consistent.

### Step 10: Clean Up

When you're done using your resources or want to delete them, Terraform provides a simple way to do so. Follow these steps to destroy the resources. First run the following command to see what changes Terraform will make to destroy the resources: `terraform plan -destroy`. This command will show you a preview of the changes that Terraform will make to destroy the resources. If everything looks correct in the plan proceed to destroy the resources: `terraform destroy`. Terraform will prompt you to confirm the destruction of resources. Type **yes** and press **Enter** to proceed. Once the destruction process is complete, Terraform will display a summary of the resources that were destroyed.

## Best Practices for Module Development

When developing Terraform modules, consider the following best practices:

- Keep modules small and focused on a specific task, making them reusable in various contexts
- Define clear and informative variable names and descriptions
- Provide sensible default values for variables
- Document module usage and input variable requirements
- Use outputs to provide information about created resources
- Follow versioning best practices to manage module versions

With these practices, your modules become more maintainable and easier to use in different projects and environments.

## Key Takeaways 

- Terraform modules allow you to encapsulate and reuse infrastructure components, promoting code modularity and reducing duplication
- Modules can accept input variables, enabling flexibility in configuring resources. Users can customize modules by providing values for these parameters.
- Modules can be composed of multiple resources, abstracting complex infrastructure components into a single, manageable unit
- Reusing modules across projects helps enforce best practices and maintain consistent configurations